In [22]:
using BSON: @save, @load
using Statistics
using DelimitedFiles
using Printf
using MDToolbox
using StatsBase 
using LinearAlgebra
using Random
using Plots

include("../src/afm.jl")

baumwelch (generic function with 1 method)

In [23]:
@load "data/viterbi/test_radius_$(25)_realization02/pred_radius_$(18)/sharpness_$(10)/q_$(576)/T_baumwelch.bson" T learned_T

In [24]:
learned_T

1246×1246 Matrix{Float64}:
 1.99265e-90   5.57675e-204  0.0952888     …  0.0  0.0  0.0
 7.20721e-203  0.0           0.0              0.0  0.0  0.0
 0.189831      0.0           1.11799e-104     0.0  0.0  0.0
 1.46111e-121  0.0           0.212328         0.0  0.0  0.0
 0.115764      0.0           5.77343e-117     0.0  0.0  0.0
 0.275202      0.156753      0.0           …  0.0  0.0  0.0
 1.57465e-174  2.18385e-178  2.48347e-76      0.0  0.0  0.0
 0.228643      0.0           1.72622e-49      0.0  0.0  0.0
 0.105055      9.9873e-277   0.13305          0.0  0.0  0.0
 0.113964      0.0           6.64748e-224     0.0  0.0  0.0
 0.104743      7.05115e-306  2.78434e-6    …  0.0  0.0  0.0
 1.0406e-82    0.00565664    0.0900361        0.0  0.0  0.0
 0.010468      5.33641e-307  0.0513835        0.0  0.0  0.0
 ⋮                                         ⋱            ⋮
 0.0           0.0           0.0              0.0  0.0  0.0
 0.0           0.0           0.0           …  0.0  0.0  0.0
 0.0           

In [25]:
@load "data/viterbi/test_radius_$(25)_realization02/pred_radius_$(15)/sharpness_$(10)/q_$(576)/rmsds_baumwelch.bson" rmsds_arr

In [26]:
@load "data/viterbi/test_radius_$(25)_realization02/pred_radius_$(15)/sharpness_$(10)/q_$(576)/rmsds_baumwelch.bson" rmsds_arr

In [27]:
rmsds_arr

576×100 Matrix{Float64}:
 53.3527  52.8722  52.594   49.1613  …  50.3333  50.3333  53.8835  52.8722
  0.0      0.0      0.0      0.0         0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0         0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0         0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0         0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0     …   0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0         0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0         0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0         0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0         0.0      0.0      0.0      0.0
  0.0      0.0      0.0      0.0     …   0.0      0.0      0.0      0.0
 52.2605  53.0408  52.3335  37.7782     46.8905  49.6567  49.2819  48.6128
  0.0      0.0      0.0      0.0         0.0      0.0      0.0      0.0
  ⋮                              

In [28]:
using DataFrames

nq = 576
test_radius = 25
pred_radii = [15, 18, 20, 25, 30, 32, 35]
#pred_radii_str = ["1.8", "2.0", "2.5", "3.0", "3.2"]
labels = ["1 frame-by-frame rigid-body fitting", "2 Viterbi with the ground-truth T", "3 Viterbi with estimated T", "4 Viterbi with random matrix"]
sigma_noise = 3
sharpness = 10
nframe = 100

plts = []
x_min = 0
x_max = 70
bin_length = 30
font_size = 10
width = (x_max - x_min) / bin_length

seed = MersenneTwister(334)
extracted_qs = sample(seed, 1:576, 50, replace=false)

rmsds = []
for r in pred_radii
    @load "data/viterbi/test_radius_$(test_radius)_realization02/pred_radius_$(r)/sharpness_$(sharpness)/q_$(nq)/rmsds_origin.bson" rmsds_arr
    @show size(rmsds_arr)
    x = DataFrame(rmsd=rmsds_arr[extracted_qs, :][:] .* 0.1, label= labels[1])
    @load "data/viterbi/test_radius_$(test_radius)_realization02/pred_radius_$(r)/sharpness_$(sharpness)/q_$(nq)/rmsds_MD.bson" rmsds_arr
    x = vcat(x, DataFrame(rmsd=rmsds_arr[extracted_qs, :][:] .* 0.1, label=labels[2]))
    @load "data/viterbi/test_radius_$(test_radius)_realization02/pred_radius_$(r)/sharpness_$(sharpness)/q_$(nq)/rmsds_baumwelch.bson" rmsds_arr
    x = vcat(x, DataFrame(rmsd=rmsds_arr[extracted_qs, :][:] .* 0.1, label=labels[3]))
    @load "data/viterbi/test_radius_$(test_radius)_realization02/pred_radius_$(r)/sharpness_$(sharpness)/q_$(nq)/rmsds_rand.bson" rmsds_arr
    x = vcat(x, DataFrame(rmsd=rmsds_arr[extracted_qs, :][:] .* 0.1, label=labels[4]))
    
    push!(rmsds, x)
end

#for i = 2:length(pred_radii)
#DataFrame
#end

size(rmsds_arr) = (576, 100)
size(rmsds_arr) = (576, 100)
size(rmsds_arr) = (576, 100)
size(rmsds_arr) = (576, 100)
size(rmsds_arr) = (576, 100)
size(rmsds_arr) = (576, 100)
size(rmsds_arr) = (576, 100)


In [29]:
df = rmsds[4]
df = filter(:label => ==("1 frame-by-frame rigid-body fitting"), df)
sum(df.rmsd .== 0.0)

4997

In [30]:
df = rmsds[4]
df = filter(:label => ==("2 Viterbi with the ground-truth T"), df)
sum(df.rmsd .== 0.0)

4997

In [31]:
df = rmsds[4]
df = filter(:label => ==("3 Viterbi with estimated T"), df)
sum(df.rmsd .== 0.0)

4997

In [32]:
rmsds[1].label

20000-element Vector{String}:
 "1 frame-by-frame rigid-body fitting"
 "1 frame-by-frame rigid-body fitting"
 "1 frame-by-frame rigid-body fitting"
 "1 frame-by-frame rigid-body fitting"
 "1 frame-by-frame rigid-body fitting"
 "1 frame-by-frame rigid-body fitting"
 "1 frame-by-frame rigid-body fitting"
 "1 frame-by-frame rigid-body fitting"
 "1 frame-by-frame rigid-body fitting"
 "1 frame-by-frame rigid-body fitting"
 "1 frame-by-frame rigid-body fitting"
 "1 frame-by-frame rigid-body fitting"
 "1 frame-by-frame rigid-body fitting"
 ⋮
 "4 Viterbi with random matrix"
 "4 Viterbi with random matrix"
 "4 Viterbi with random matrix"
 "4 Viterbi with random matrix"
 "4 Viterbi with random matrix"
 "4 Viterbi with random matrix"
 "4 Viterbi with random matrix"
 "4 Viterbi with random matrix"
 "4 Viterbi with random matrix"
 "4 Viterbi with random matrix"
 "4 Viterbi with random matrix"
 "4 Viterbi with random matrix"

In [33]:
using StatsPlots
ps = []

for i in 1:length(pred_radii)
    p = @df rmsds[i] groupedhist(:rmsd, group=:label, bar_position=:dodge, 
    bar_width=0.4, bins=20, framestyle=:box, size=(1100, 250), dpi=300, 
    xlim=(-0.3, 8), #ylim=(0, 3000), 
    #labels=map(x -> x[3:end], labels), 
    label=nothing, 
    xlabel="RMSD [nm]", ylabel="frequency", 
    legend=:top, 
    left_margin=Plots.Measures.Length(:mm, 10.0),
    bottom_margin=Plots.Measures.Length(:mm, 10.0))
    push!(ps, p)
end

In [34]:
for i in 1:length(ps)
    savefig(ps[i], "fig03_$(pred_radii[i])A_realization02.png")
end